# STSWebscrape

In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import re
import pandas as pd

In [2]:
#retrieve the HTML content of the webpage located at the URL 'https://publicreporting.sts.org/search/acsd'.
html = urlopen('https://tvtpublicreporting.sts.org/tvt-registry')

# creates a BeautifulSoup object bs that can be used to parse and manipulate the HTML content.
bs = BeautifulSoup(html.read(), 'html.parser')

In [4]:
# regular expression pattern to match '/participant/' followed by any number of digits
pattern = re.compile(r'/site/\d+')

# find all 'a' tags with 'href' attribute matching the regular expression pattern
bs.find_all('a', href=pattern)

[<a href="/site/772319" hreflang="en">AdventHealth Ocala</a>,
 <a href="/site/456326" hreflang="en">AdventHealth Tampa, Pepin Heart Institute</a>,
 <a href="/site/961195" hreflang="en">AdventHealth Waterman</a>,
 <a href="/site/654912" hreflang="en">Adventist Health Glendale</a>,
 <a href="/site/554000" hreflang="en">ADVENTIST HEALTH ST. HELENA</a>,
 <a href="/site/884031" hreflang="en">Adventist Healthcare White Oak Medical Center</a>,
 <a href="/site/856498" hreflang="en">Advocate Aurora St. Luke's Medical Center</a>,
 <a href="/site/487702" hreflang="en">Advocate Good Samaritan Hospital</a>,
 <a href="/site/373631" hreflang="en">Advocate Good Shepherd Hospital</a>,
 <a href="/site/102309" hreflang="en">Advocate Illinois Masonic Medical Center</a>,
 <a href="/site/930946" hreflang="en">Advocate Lutheran General Hospital</a>,
 <a href="/site/194521" hreflang="en">Advocate Sherman Hospital</a>,
 <a href="/site/303521" hreflang="en">Albany Medical Center Hospital</a>,
 <a href="/site/61

Here we return a list of all a tags whose href attribute matches the regular expression pattern /participant/\d+, where \d+ matches one or more digits.

In [12]:
# find all 'a' tags with 'href' attribute matching the regular expression pattern
links = bs.find_all('a', href=pattern)

# scrape content from the first link
if links:
    link = links[0]['href']
    
    # Extract digits from the link
    digits = re.findall(r'\d+', link)
    pid = int(digits[0]) if digits else None

    
    url = f'https://tvtpublicreporting.sts.org/{link}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # extract information from the scraped page using BeautifulSoup

    hospital = soup.find('h1').text.strip()
    location = soup.find('div', {'class':'views-field views-field-field-address' }).text.strip()
    composite_score = soup.find('div', {'class': 'value'}).text.strip()
    confidence_interval = soup.find('div', {'class': 'interval'}).text.strip()
    star_rating_varchar = soup.find('div', {'class': 'status'}).text.strip()  

    procedure_timeframe_div = soup.find('div', {'class': 'views-field views-field-field-reporting-period'})
    first_tavr_div = soup.find('div', {'class': 'views-field views-field-field-date-of-first-tavr-procedu'})
    cum_vol_div = soup.find('div', {'class': 'views-field views-field-field-cumulative-tavr-volume'})
    reporting_vol_div = soup.find('div', {'class': 'views-field views-field-field-eligible-procedures-in-rep'})
    avg_annual_vol_div = soup.find('div', {'class': 'views-field views-field-field-annual-tavr-volume'})



# Check if the div exists and then find the inner div with the actual timeframe.
    if procedure_timeframe_div:
        procedure_timeframe = procedure_timeframe_div.find('div', {'class': 'field-content'}).text.strip()
    else:
        procedure_timeframe = None

# Check if the div exists and then find the inner div with the actual timeframe.
    if cum_vol_div:
        cum_vol = cum_vol_div.find('div', {'class': 'field-content'}).text.strip()
    else:
        cum_vol = None

# Check if the div exists and then find the inner div with the actual timeframe.
    if first_tavr_div:
        first_tavr = first_tavr_div.find('div', {'class': 'field-content'}).text.strip()
    else:
        first_tavr = None

# Check if the div exists and then find the inner div with the actual timeframe.
    if reporting_vol_div:
        reporting_vol = reporting_vol_div.find('div', {'class': 'field-content'}).text.strip()
    else:
        reporting_vol = None

# Check if the div exists and then find the inner div with the actual timeframe.
    if avg_annual_vol_div:
        avg_annual_vol = avg_annual_vol_div.find('div', {'class': 'field-content'}).text.strip()
    else:
        avg_annual_vol = None

    # print the scraped information
    print(pid)
    print(hospital)
    print(location)
    print(procedure_timeframe)
    print(composite_score)
    print(confidence_interval)
    print(star_rating_varchar)
    print(first_tavr)
    print(cum_vol)
    print(reporting_vol)
    print(avg_annual_vol)
    


772319
AdventHealth Ocala
Ocala, FL
October 1, 2019 through September 30, 2022
0.01
(-0.02, 0.04)
As Expected
Nov, 2012
583
273
93


In [14]:
#loop through links on first page
# regular expression pattern to match '/participant/' followed by any number of digits
pattern = re.compile(r'/site/\d+')

# create an empty list to store the dataframes
dfs = []

# loop through all links with the pattern
for link in bs.find_all('a', href=pattern):
    url = f'https://tvtpublicreporting.sts.org/{link["href"]}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract digits from the link
    digits = re.findall(r'\d+', link['href'])
    pid = int(digits[0]) if digits else None

    # extract hospital and location
    hospital = soup.find('h1').text.strip()
    location = soup.find('div', {'class':'views-field views-field-field-address' }).text.strip()


    try:
        def extract_text(soup, outer_class, inner_class='field-content'):
            outer_div = soup.find('div', {'class': outer_class})
            if outer_div:
                inner_div = outer_div.find('div', {'class': inner_class})
                return inner_div.text.strip() if inner_div else None
            return None

        # Using the function to extract the required information.
        composite_score = soup.find('div', {'class': 'value'}).text.strip()
        confidence_interval = soup.find('div', {'class': 'interval'}).text.strip()
        star_rating_varchar = soup.find('div', {'class': 'status'}).text.strip()  
        procedure_timeframe = extract_text(soup, 'views-field views-field-field-reporting-period')
        cum_vol = extract_text(soup, 'views-field views-field-field-cumulative-tavr-volume')
        first_tavr = extract_text(soup, 'views-field views-field-field-date-of-first-tavr-procedu')
        reporting_vol = extract_text(soup, 'views-field views-field-field-eligible-procedures-in-rep')
        avg_annual_vol = extract_text(soup, 'views-field views-field-field-annual-tavr-volume')


        # create a dictionary from the scraped information
        data = {'pid': [pid], 'hospital': [hospital], 'location': [location], 'composite_score': [composite_score], 'confidence_interval': [confidence_interval], 'star_rating_varchar': [star_rating_varchar],
                'procedure_timeframe': [procedure_timeframe], 'cum_vol': [cum_vol], 'first_tavr': [first_tavr], 
                'reporting_vol': [reporting_vol], 'avg_annual_vol': [avg_annual_vol]}

        # create a dataframe from the dictionary and append to the list of dataframes
        dfs.append(pd.DataFrame(data))
    except AttributeError:
        # skip the page if the desired data is not found
        continue

# concatenate all dataframes in the list into one dataframe
result = pd.concat(dfs, ignore_index=True)

print(result)

       pid                                           hospital  \
0   772319                                 AdventHealth Ocala   
1   456326          AdventHealth Tampa, Pepin Heart Institute   
2   856498          Advocate Aurora St. Luke's Medical Center   
3   487702                   Advocate Good Samaritan Hospital   
4   373631                    Advocate Good Shepherd Hospital   
5   930946                 Advocate Lutheran General Hospital   
6   194521                          Advocate Sherman Hospital   
7   303521                     Albany Medical Center Hospital   
8   615332                    Alexian Brothers Medical Center   
9   949340                         Allegheny General Hospital   
10  874649    Allina Health Minneapolis Heart Institute (ANW)   
11  208404  Allina Health Minneapolis Heart Institute (Uni...   
12  267989                   Alta Bates Summit Medical Center   
13  745175                         Ascension Borgess Hospital   
14  325824               

This modified code will iterate through all links with the specified pattern and scrape data from each one. It will then append the scraped data to a list called data_list. After scraping all the links, it will create a dataframe from the data_list and export it to a CSV file. Note that the index=False argument in the to_csv function call is used to prevent the index column from being included in the exported CSV file. This will skip any sites that do not have a rating. Will skip any non-CABG ratings

# Putting Everything Together

In [15]:
# create an empty list to store the dataframes
dfs = []

# Loop through pages
page = 0
while True:
        
    # Get the page content
    base_url = f'https://tvtpublicreporting.sts.org/tvt-registry?page={page}'
    response = requests.get(base_url)
    bs = BeautifulSoup(response.content, 'html.parser')

    # regular expression pattern to match '/site/' followed by any number of digits
    pattern = re.compile(r'/site/\d+')

        # Loop through all links with the pattern
    links = bs.find_all('a', href=pattern)
    
    # If no links are found on the current page, exit the loop
    if not links:
        break
    
    for link in links:
        url = f'https://tvtpublicreporting.sts.org/{link["href"]}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        print(f'Processing link: {url}')
        
        # Extract digits from the link
        digits = re.findall(r'\d+', link['href'])
        pid = int(digits[0]) if digits else None
        # loop through all links with the pattern


        # extract hospital and location
        hospital = soup.find('h1').text.strip()
        location = soup.find('div', {'class':'views-field views-field-field-address' }).text.strip()


        try:
            def extract_text(soup, outer_class, inner_class='field-content'):
                outer_div = soup.find('div', {'class': outer_class})
                if outer_div:
                    inner_div = outer_div.find('div', {'class': inner_class})
                    return inner_div.text.strip() if inner_div else None
                return None

            # Using the function to extract the required information.
            composite_score = soup.find('div', {'class': 'value'}).text.strip()
            confidence_interval = soup.find('div', {'class': 'interval'}).text.strip()
            star_rating_varchar = soup.find('div', {'class': 'status'}).text.strip()  
            procedure_timeframe = extract_text(soup, 'views-field views-field-field-reporting-period')
            cum_vol = extract_text(soup, 'views-field views-field-field-cumulative-tavr-volume')
            first_tavr = extract_text(soup, 'views-field views-field-field-date-of-first-tavr-procedu')
            reporting_vol = extract_text(soup, 'views-field views-field-field-eligible-procedures-in-rep')
            avg_annual_vol = extract_text(soup, 'views-field views-field-field-annual-tavr-volume')


            # create a dictionary from the scraped information
            data = {'pid': [pid], 'hospital': [hospital], 'location': [location], 'composite_score': [composite_score], 'confidence_interval': [confidence_interval], 'star_rating_varchar': [star_rating_varchar],
                    'procedure_timeframe': [procedure_timeframe], 'cum_vol': [cum_vol], 'first_tavr': [first_tavr], 
                    'reporting_vol': [reporting_vol], 'avg_annual_vol': [avg_annual_vol]}

            # create a dataframe from the dictionary and append to the list of dataframes
            dfs.append(pd.DataFrame(data))
        except AttributeError:
            # skip the page if the desired data is not found
            continue

    # Increment the page number for the next iteration
    print(f'Finished processing page {page}')
    page += 1
            

if dfs:
    result = pd.DataFrame(dfs)

else:
    print("No data found.")

# export the dataframe to a CSV file
result.to_csv('TVTWebscrape.csv', index = False)

Processing link: https://tvtpublicreporting.sts.org//site/772319
Processing link: https://tvtpublicreporting.sts.org//site/456326
Processing link: https://tvtpublicreporting.sts.org//site/961195
Processing link: https://tvtpublicreporting.sts.org//site/654912
Processing link: https://tvtpublicreporting.sts.org//site/554000
Processing link: https://tvtpublicreporting.sts.org//site/884031
Processing link: https://tvtpublicreporting.sts.org//site/856498
Processing link: https://tvtpublicreporting.sts.org//site/487702
Processing link: https://tvtpublicreporting.sts.org//site/373631
Processing link: https://tvtpublicreporting.sts.org//site/102309
Processing link: https://tvtpublicreporting.sts.org//site/930946
Processing link: https://tvtpublicreporting.sts.org//site/194521
Processing link: https://tvtpublicreporting.sts.org//site/303521
Processing link: https://tvtpublicreporting.sts.org//site/615332
Processing link: https://tvtpublicreporting.sts.org//site/949340
Processing link: https://

AttributeError: 'NoneType' object has no attribute 'text'

In [18]:
#modified using extract_text to ensure absence of elements don't cause code to fail
#concat dfs before csv, error in original script

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Function to safely extract text
def extract_text(soup, outer_class, inner_class='field-content'):
    outer_div = soup.find('div', {'class': outer_class})
    if outer_div:
        inner_div = outer_div.find('div', {'class': inner_class})
        return inner_div.text.strip() if inner_div else None
    return None

# Create an empty list to store the dataframes
dfs = []

# Loop through pages
page = 0
while True:
    # Get the page content
    base_url = f'https://tvtpublicreporting.sts.org/tvt-registry?page={page}'
    response = requests.get(base_url)
    bs = BeautifulSoup(response.content, 'html.parser')

    # Regular expression pattern to match '/site/' followed by any number of digits
    pattern = re.compile(r'/site/\d+')

    # Loop through all links with the pattern
    links = bs.find_all('a', href=pattern)

    # If no links are found on the current page, exit the loop
    if not links:
        break


    def safe_extract_text(element):
        return element.text.strip() if element else None

    for link in links:
        url = f'https://tvtpublicreporting.sts.org/{link["href"]}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        print(f'Processing link: {url}')

        # Extract digits from the link
        digits = re.findall(r'\d+', link['href'])
        pid = int(digits[0]) if digits else None

        # Extract hospital and location
        hospital = soup.find('h1').text.strip() if soup.find('h1') else None
        location = extract_text(soup, 'views-field views-field-field-address')

        # Directly extract text for certain fields with a check for None
        composite_score_element = soup.find('div', {'class': 'value'})
        confidence_interval_element = soup.find('div', {'class': 'interval'})
        star_rating_varchar_element = soup.find('div', {'class': 'status'})

        composite_score = safe_extract_text(composite_score_element)
        confidence_interval = safe_extract_text(confidence_interval_element)
        star_rating_varchar = safe_extract_text(star_rating_varchar_element)
 

        procedure_timeframe = extract_text(soup, 'views-field views-field-field-reporting-period')
        cum_vol = extract_text(soup, 'views-field views-field-field-cumulative-tavr-volume')
        first_tavr = extract_text(soup, 'views-field views-field-field-date-of-first-tavr-procedu')
        reporting_vol = extract_text(soup, 'views-field views-field-field-eligible-procedures-in-rep')
        avg_annual_vol = extract_text(soup, 'views-field views-field-field-annual-tavr-volume')

        # Create a dictionary from the scraped information
        data = {'pid': pid, 'hospital': hospital, 'location': location, 'composite_score': composite_score, 'confidence_interval': confidence_interval, 'star_rating_varchar': star_rating_varchar,
                'procedure_timeframe': procedure_timeframe, 'cum_vol': cum_vol, 'first_tavr': first_tavr,
                'reporting_vol': reporting_vol, 'avg_annual_vol': avg_annual_vol}

        # Create a dataframe from the dictionary and append to the list of dataframes
        dfs.append(pd.DataFrame([data]))

    # Increment the page number for the next iteration
    print(f'Finished processing page {page}')
    page += 1

# Concatenate all dataframes in the list
if dfs:
    result = pd.concat(dfs, ignore_index=True)
    # Export the dataframe to a CSV file
    result.to_csv('TVTWebscrape.csv', index=False)
    print("Data exported to TVTWebscrape.csv.")
else:
    print("No data found.")


Processing link: https://tvtpublicreporting.sts.org//site/772319
Processing link: https://tvtpublicreporting.sts.org//site/456326
Processing link: https://tvtpublicreporting.sts.org//site/961195
Processing link: https://tvtpublicreporting.sts.org//site/654912
Processing link: https://tvtpublicreporting.sts.org//site/554000
Processing link: https://tvtpublicreporting.sts.org//site/884031
Processing link: https://tvtpublicreporting.sts.org//site/856498
Processing link: https://tvtpublicreporting.sts.org//site/487702
Processing link: https://tvtpublicreporting.sts.org//site/373631
Processing link: https://tvtpublicreporting.sts.org//site/102309
Processing link: https://tvtpublicreporting.sts.org//site/930946
Processing link: https://tvtpublicreporting.sts.org//site/194521
Processing link: https://tvtpublicreporting.sts.org//site/303521
Processing link: https://tvtpublicreporting.sts.org//site/615332
Processing link: https://tvtpublicreporting.sts.org//site/949340
Processing link: https://